<a href="https://colab.research.google.com/github/iesous-kurios/family_promise_data/blob/master/Last_Updated_YTD_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **YTD FPS Data Notebook**

In [1]:
#@title
# mount google drive to access weekly data
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#@title
# import libraries which provide math functions to our data imports
import statistics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xlwt 
from xlwt import Workbook
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib
from matplotlib import style

sns.__version__

'0.9.0'

In [0]:
#@title
# import weekly data from yearly FPS folder
df = pd.read_excel ('/content/Nov_6_YTD.xlsx')

In [0]:
#@title
# replace zeros in data with "Nan" to calculate more accurate average bednights 
df['Count of Bed Nights (Housing Check-ins)'] = df['Count of Bed Nights (Housing Check-ins)'].replace(0, np.NaN)

In [5]:
# display all possible exit destinations and their totals
df['3.12 Exit Destination'].value_counts()

Client refused                                                                                                                   286
Emergency shelter, including hotel or motel paid for with emergency shelter voucher, or RHY-funded Host Home shelter             126
Rental by client with RRH or equivalent subsidy                                                                                  102
No exit interview completed                                                                                                       59
Transitional Housing for homeless persons (including homeless youth)                                                              37
Rental by client, no ongoing housing subsidy                                                                                      36
Staying or living with family, permanent tenure                                                                                   29
Staying or living with family, temporary tenure (e.g., room, apartmen

In [0]:
#@title

exit_reasons = ['Rental by client with RRH or equivalent subsidy', 
                'Rental by client, no ongoing housing subsidy', 
                'Staying or living with family, permanent tenure', 
                'Rental by client, other ongoing housing subsidy',
                'Permanent housing (other than RRH) for formerly homeless persons', 
                'Staying or living with friends, permanent tenure', 
                'Owned by client, with ongoing housing subsidy', 
                'Staying or living with friends, permanent tenure', 
                'Rental by client, VASH housing Subsidy'
               ]

temp_reasons = ['Staying or living with family, temporary tenure (e.g., room, apartment or house)', 
               'Staying or living with friends, temporary tenure (e.g., room, apartment or house)', 
                
               ]

In [7]:
#@title  { display-mode: "both" }
# pull all exit destinations from main data file and sum up the totals of each destination, placing them into new df for calculations
exits = df['3.12 Exit Destination'].value_counts()

tempExits = df['3.12 Exit Destination'].isin(temp_reasons)
temp_exit_percent = tempExits.sum() / exits.sum()
#exits.sum()) * 100)
# round tempExits to two decimals
tempExits = round(temp_exit_percent*100, 2)
#tempExits = 0

# take sum of unknown exits and divide by total exits, then multiply by 100 to get percentage
unknownExits = ((exits['Client refused'] / exits.sum()) * 100)

# round the answer to nearest hundreth place
unknownExits = (round(unknownExits, 2))

# create three data frames made up of the three key words found in perm housing
perm1 = df[df['3.12 Exit Destination'].str.contains('Rental') == True] 
perm2 = df[df['3.12 Exit Destination'].str.contains('permanent') == True]
perm3 = df[df['3.12 Exit Destination'].str.contains('Permanent') == True]

# merge the three dataframes into one to calculate exit to perm percent
perm_almost_final = perm1.append(perm2)
perm_final = perm_almost_final.append(perm3)
perm_final['3.12 Exit Destination'].value_counts()
perm_final_sums = perm_final['3.12 Exit Destination'].value_counts()
perm_final = df['3.12 Exit Destination'].isin(exit_reasons)
# calculate percentage of exits to perm
perm_final_percent = perm_final_sums.sum() / exits.sum()

#perm_final.sum()

#perm_final_percent2 = perm_final.sum() / df['3.12 Exit Destination'].value_counts().sum()
#round(perm_final_percent1, 2)
print('Exit to Perm percent:',perm_final_percent*100,'%')
print('Unknown Exit percent:',unknownExits,'%')
print('Temp Exit percent:',tempExits,'%')

Exit to Perm percent: 25.0 %
Unknown Exit percent: 38.65 %
Temp Exit percent: 4.59 %


In [8]:
#@title
# take sum of ES and Trans exits and divide by total exits, then multiply by 100 to get percentage
esExits = (( exits['Emergency shelter, including hotel or motel paid for with emergency shelter voucher, or RHY-funded Host Home shelter'] / exits.sum()) * 100)
transExits = ((exits['Transitional Housing for homeless persons (including homeless youth)'] / exits.sum()) * 100)
# round the answer to nearest hundreth place
esExits = (round(esExits, 2))
transExits = (round(transExits, 2))

print('Exits to ES percent:',esExits,'%')
print('Exits to Transitional',transExits,'%')

Exits to ES percent: 17.03 %
Exits to Transitional 5.0 %


In [9]:
#@title

avgBedNights = df['Count of Bed Nights (Housing Check-ins)'].mean(skipna=True)

avgBedNights = round(avgBedNights, 2)

print("Average BedNights is:", avgBedNights)

Average BedNights is: 27.94


In [10]:
#@title
avgDaysEnrolled = df['Days Enrolled in Project'].mean(skipna=True)
avgDaysEnrolled = round(avgDaysEnrolled, 2)                   
perm_final_percent = round(perm_final_percent, 2)
print('Average Days Enrolled is:', avgDaysEnrolled)

Average Days Enrolled is: 42.83


In [11]:
#@title
print('Average Household Income at Enrollment is:', round(df['4.2 Income Total at Entry'].mean(skipna=True), 2))

print('Average Household Income at Exit is:', round(df['4.2 Income Total at Exit'].mean(skipna=True), 2))

avgChange = (df['4.2 Income Total at Exit'].mean(skipna=True)) - (df['4.2 Income Total at Entry'].mean(skipna=True))
avgChange = round(avgChange, 2)
print('Average Change in Income for Households during stay is:', avgChange)


Average Household Income at Enrollment is: 922.95
Average Household Income at Exit is: 953.2
Average Change in Income for Households during stay is: 30.25


In [12]:
#@title
# printed rounded answer for average age during enrollment
print('Average Age of Guest during Enrollment is:', round(df['Age at Enrollment'].mean(skipna=True), 2))

Average Age of Guest during Enrollment is: 19.35


In [0]:
# export zips into an excel sheet for Power BI visual
  df['V5 Zip'].value_counts().to_excel('/content/drive/My Drive/FPS Weekly Data Colabs/YTD Data/weeklyYTDzips.xlsx', header=True, index=True)

In [0]:
# create list of all options for engagement data and export into excel file for Power BI
df['Days Enrolled Until Engagement Date'].value_counts().to_excel('/content/drive/My Drive/FPS Weekly Data Colabs/YTD Data/weeklyYTD_aggregate_DOE.xlsx', header=True, index=True)

In [0]:
# create new excel sheet with the days enrolled until engagement to upload back into notebook for calculating mean

df['Days Until Engagement'] = (df['4.13 Engagement Date'] - df['3.10 Enroll Date'])
df['Days Until Engagement'].mean()
d_o_e_avg = df['Days Until Engagement'].mean()

In [16]:
d_o_e_avg

Timedelta('4 days 11:54:02.975206')

In [0]:
# create new excel sheet with the days enrolled until engagement to upload back into notebook for calculating mean 
(df['4.13 Engagement Date'] - df['3.10 Enroll Date']).to_excel('/content/drive/My Drive/FPS Weekly Data Colabs/YTD Data/ytdDOE.xlsx', header=False, index=False)

In [0]:
# create new column with total amount of guests for year
df['Total Guests'] = df['5.8 Personal ID'].drop_duplicates().count()

In [0]:
# create new column with count of unique households
df['Total Households'] = df['5.9 Household ID'].drop_duplicates().count()

# create new column with total amount of guests for year
df['Total Guests'] = df['5.8 Personal ID'].drop_duplicates().count()


In [0]:
import plotly.express as px

In [21]:
totalHouseholds = df.iloc[1, 120]
totalGuests = df.iloc[1, 119]

print(totalHouseholds)
print(totalGuests)

258
766


In [0]:
# Workbook is created 
wb = Workbook() 
  
# add_sheet is used to create sheet. 
sheet1 = wb.add_sheet('Sheet 1') 
  
sheet1.write(0, 0, 'Permanent Housing') 
sheet1.write(0, 1, round(perm_final_percent*100,2))
sheet1.write(1, 0, 'Emergency Shelter') 
sheet1.write(1, 1, esExits) 
sheet1.write(2, 0, 'Temporary Housing') 
sheet1.write(2, 1, tempExits) 
sheet1.write(3, 0, 'Transitional Housing') 
sheet1.write(3, 1, transExits) 
sheet1.write(4, 0, 'Guest left without completing exit survey') 
sheet1.write(4, 1, unknownExits) 
sheet1.write(5, 0, 'Average Bednights')
sheet1.write(5, 1, avgBedNights)
sheet1.write(6, 0, 'Average Days Enrolled')
sheet1.write(6, 1, avgDaysEnrolled)
sheet1.write(7, 0, 'Average Change in Income per Household')
sheet1.write(7, 1, avgChange)
sheet1.write(8, 0, 'Total Households Served')
sheet1.write(8, 1, int(totalHouseholds))
sheet1.write(9, 0, 'Total Guests Served')
sheet1.write(9, 1, int(totalGuests))

wb.save('/content/drive/My Drive/FPS Weekly Data Colabs/YTD Data/weeklyYtdData.xls')